# Introduction to SQL

[SQL](https://en.wikipedia.org/wiki/SQL) stands for Structured Query Language (you can pronounce it as 'sequel' or 'ess-cue-elle'). It allows you to retrieve data stored in relational databases and manipulate it in a variety of ways.

If a concept of a database is new to you, you can think of it as a collection of tables. For instance, you can draw a comparison to an Excel workbook with multiple worksheets where each worksheet is somehow related to another. Worksheets in this comparison are SQL tables and the entire workbook would be a database. The key is to have some sort of relationship between the tables that allows you to bring all that data together (this is just an analogy and Excel workbook is not a database!).

Each table in SQL consists of rows and columns where columns represent different data attributes and rows represent observations or data records. Tables can contain different types of data but the main types are integer, decimal, character or string, date, and time.

There are many DB systems available like MySQL, PostgreSQL, Oracle Database, MS SQL Server, Amazon Redshift, etc. However, all of them speak SQL, so once you've got the hang of the basic SQL syntax you'll be able to work with any of them.

To keep things simple, we will be using a database system called [SQLite](https://www.sqlite.org/about.html) to practice SQL syntax.
Unlike most other SQL databases, SQLite does not have a separate server. SQLite reads and writes directly to ordinary files on your computer. A complete SQL database with multiple tables, etc., is contained in a single file.

### Resources

Since SQL is the main database language used worldwide, there are plenty of resources to master it:

- https://sqlzoo.net/

- https://sqlbolt.com

- https://leetcode.com

- https://www.sql-ex.ru/learn_exercises.php

# Using Raw SQL cursor connections

In [ ]:
import pandas as pd
import sqlite3
conn = sqlite3.connect('../data/mtcars.sqlite') # create a connection
c = conn.cursor() # create a cursor (a mechanism that enables traversal over the records in a database)

### SELECT

SELECT is the most important command as it allows you to retrieve data from a table. SELECT on itself won't do anything, so you need to specify what data you want to retrieve and from where.

You can retrieve data from a table, so you need to specify a table name in the SELECT statement. SELECT statement refers to the columns part of a table and not the rows, so you also need to list columns. Here are a few things to remember:

- To select a column, you will simply need to type its name
- You can select all columns by typing their name or you can use * to select all columns without typing their names
- You can select as many or as few columns from a table as you want
- You can even create new columns in the SELECT statement


You can even create new columns in the SELECT statement. SQL supports mathematical operations and has a variety of built-in functions https://www.sqlitetutorial.net/


In [ ]:
# OPTION 1
# we can use sqlite3 directly
cursor = c.execute(f"SELECT * FROM results")
row = cursor.fetchall()

In [ ]:
row[0:2] # gets data back as a list of tuples

In [ ]:
# get column names
column_names = list(map(lambda x: x[0], cursor.description))

print(column_names)

In [ ]:
# OPTION 2
# we can also use pandas
df = pd.read_sql_query("SELECT * FROM results", conn) # pass query & connection
df.head()

In [ ]:
# select only mpg & cylinders
pd.read_sql_query("SELECT mpg, cylinders, name FROM results", conn)

## WHERE

There will be cases where you don't want all of the observations to be returned; that's when you use the WHERE clause. It is typically specified after a table name (unless you do a join, more on this to come). This is a typical syntax <code> WHERE column_name operator value </code>


- Mathematical comparisons with the following operators =, <,>, <=, >=, <> . Note that <> means not equal. =, <> can be used for strings and numbers
- To search for a pattern in a string, you can use LIKE operator
- You can use IN operator to specify multiple values in a column
- You can have multiple filters in the WHERE clause separated by AND or OR. Don't forget to specify column name for every filter you pass in the WHERE clause


In [ ]:
pd.read_sql_query("""SELECT * 
                        FROM results
                        WHERE mpg < 20 AND cylinders in (6,8) AND year <> 70 AND
                              name LIKE '%ford%'
                   """, conn) # I like to use """ instead of ' or "

In [ ]:
pd.read_sql_query("""SELECT * 
                        FROM results
                        WHERE cylinders = 6 OR 8
                   """, conn) #Notice it doesn't work as expected

In [ ]:
pd.read_sql_query("""SELECT * 
                        FROM results
                        WHERE cylinders = 6 OR cylinders = 8
                   """, conn) # We needs to be specific. 

In [ ]:
pd.read_sql_query("""SELECT * 
                        FROM results
                        WHERE cylinders = 6 OR cylinders = 8 AND year <> 70
                   """, conn)

## CASE WHEN

You can use conditional logic to create variables by following this syntax.

<code> CASE WHEN condition THEN result1 ELSE result2 END AS new_variable </code>

You can also have multiple conditions.

<code> CASE WHEN condition1 THEN result1 WHEN condition2 THEN result2 ELSE result3 END AS new_variable  </code>

In [ ]:
pd.read_sql_query("""
                    SELECT *,
                    CASE WHEN mpg < 20 THEN '<20' ELSE '20+' END AS mpg_gp
                    FROM results""", conn)

## SQL TABLE

### CREATE
The CREATE TABLE command creates a new table in your database. You can either create temporary or permanent tables. You can also create a table based on an existing table or you can create an empty table and populate it. 

Temporary table syntax varies from one DB system to another. We'll look at SQLite syntax, but here is an example of MS SQL Server syntax:

<code> SELECT * INTO #temp_tbl_nm FROM existing_tbl </code>

SQLite syntax:

<code> CREATE *TEMPORARY* TABLE temp_table AS SELECT * FROM existing_tbl </code>

Syntax to create an empty table:

<code> CREATE TABLE new_table ( column1 int, column2 varchar(255)) </code>

### INSERT INTO

To add rows into a table, we can use the INSERT INTO statement. 

<code> INSERT INTO tbl_name (column1, column2) VALUES (1, 'Canada') </code>

### ALTER
If we want to mofidy a column in a table (add, delete, change data type, etc.), we can use the ALTER TABLE command.

<code> ALTER TABLE new_table ADD column3 varchar(5) </code>

### DROP 

The DROP TABLE command deletes a table in a database. It can be applied to both permanent and temporary tables.

<code> DROP TABLE tbl_name </code>


## Making a new table in Python

You can create a new SQL table from an existing data frame in Python

In [ ]:
df

In [ ]:
df.to_sql(
    name="test_output",
    con=conn, 
    schema=None, 
    if_exists='replace', 
    index=True, 
)

In [ ]:
pd.read_sql("SELECT * FROM test_output LIMIT 10", con=conn)
# limit 10 works just like .head(10)
# in MS SQL Server, SELECT top 10 * FROM test_output
# in Oracle, SELECT * FROM test_output WHERE rownum <=10

### List  tables in a database

Table and index names can be listed by doing a **SELECT** on a special table named "***SQLITE_MASTER***". Every SQLite database has an SQLITE_MASTER table that defines the schema for the database. For tables, the ***type*** field will always be '***table***' and the name field will be the name of the table. So to get a list of all tables in the database, use the following SELECT command:

See more at https://www.sqlite.org/faq.html#q7.

In [ ]:
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", con=conn)

# Making a new DB

Is as simple as connecting to a new file name

In [ ]:
song = pd.read_csv('../data/song_data.csv')

In [ ]:
conn = sqlite3.connect('song.sqlite')
song.to_sql('song_data',con=conn,index=False,if_exists='replace')
    

In [ ]:
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'",conn)

In [ ]:
pd.read_sql_query("""SELECT * FROM song_data""",conn)